In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
root = os.path.join("..","Data")
df = pd.read_excel(os.path.join(root, "DEMAND_FILE_PHASE2.xlsx"))
df = df.iloc[0:2].T.reset_index(drop=True)
df.columns = ['Date','Sales']
df.drop(0, inplace=True)
df['Sales'] = df['Sales'].astype('float')
df['Date'] = pd.to_datetime(df['Date'])


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

def create_sliding_window_sets(df, window_size=30):
    sales = df['Sales'].values
    dates = df['Date'].values
    X, Y, X_dates = [], [], []

    for i in range(window_size, len(sales) - window_size):
        x_window = sales[i - window_size:i]         # previous 30 days
        y_window = sales[i + 1:i + 1 + window_size] # next 30 days
        today_date = dates[i]                       # today

        X.append(x_window)
        Y.append(y_window)
        X_dates.append(today_date)

    return np.array(X), np.array(Y), np.array(X_dates)

def mape(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # Avoid division by zero by adding a small epsilon where y_true is zero
    epsilon = tf.keras.backend.epsilon()
    y_true = tf.where(tf.equal(y_true, 0), epsilon, y_true)

    return tf.reduce_mean(tf.abs((y_true - y_pred) / y_true)) * 100

In [43]:
def train_model(num_days,X_train, Y_train):
    # Build FFNN model
    model = models.Sequential([
        layers.Input(shape=(num_days,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_days) 
    ])
    early_stopping = EarlyStopping(
    monitor='mape',
    mode = 'min',
    patience=15,
    restore_best_weights=True )

    model.compile(optimizer='adam', loss='mse', metrics=[mape])

    # Train the model
    history = model.fit(X_train, Y_train, epochs=50, batch_size=4, verbose=1,callbacks=[early_stopping])

    return model, history

In [53]:
num_days = 30
dates_list = [f"2024-{i}-01" for i in [f'0{j}' for j in range(1,10)] + ['10','11','12']]+["2025-01-01"]
pred_df = pd.DataFrame(columns=['Date','MAPE'])
for i in range(len(dates_list)-3):
    train_df = df[df['Date']<dates_list[i]].reset_index(drop=True)
    test_df = df[(df['Date'] >= dates_list[i]) & (df['Date'] < dates_list[i+3])].reset_index(drop=True)
    X_train, Y_train, dates_train = create_sliding_window_sets(train_df,num_days)
    X_test, Y_test, dates_test = create_sliding_window_sets(test_df,num_days)

    model, _ = train_model(num_days,X_train, Y_train)
    Y_pred = model.predict(X_test, verbose=0)

    mapes=[]
    for i in range(len(Y_pred)):
        mapes.append(mape(Y_pred[i],Y_test[i]))
    mapes=np.array(mapes)
    dates = pd.date_range(start=dates_test[0], periods=len(X_test))
    mape_df = pd.DataFrame({'Date':dates,'MAPE': mapes})
    pred_df = pd.concat([pred_df,mape_df])
    

Epoch 1/50
327/327 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 13125072896.0000 - mape: 34.7110
Epoch 2/50
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3032619264.0000 - mape: 21.3645
Epoch 3/50
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2662833152.0000 - mape: 19.6014
Epoch 4/50
327/327 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2924021760.0000 - mape: 21.5708
Epoch 5/50
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2541294848.0000 - mape: 19.5319
Epoch 6/50
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2734130176.0000 - mape: 20.7617
Epoch 7/50
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2594482176.0000 - mape: 19.8202
Epoch 8/50
327/327 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2568356864.0000 - mape: 19.9843
Epoch 9/50
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2670680832.0000 - mape: 20.2696
Epoch 10/50
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2644154624.0000 - mape: 19.7884
Epoch 11/50
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2616439808.0000 -

C:\Users\milan\AppData\Local\Temp\ipykernel_16088\2041930579.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pred_df = pd.concat([pred_df,mape_df])


335/335 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 11861892096.0000 - mape: 33.6197
Epoch 2/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2584982016.0000 - mape: 19.7317
Epoch 3/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2642264320.0000 - mape: 20.7062
Epoch 4/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2708081408.0000 - mape: 20.4983
Epoch 5/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2508039168.0000 - mape: 19.3352
Epoch 6/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2670692352.0000 - mape: 20.3282
Epoch 7/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2565753344.0000 - mape: 19.4048
Epoch 8/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2729252864.0000 - mape: 20.8103
Epoch 9/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2567721984.0000 - mape: 19.5114
Epoch 10/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2561016576.0000 - mape: 19.8153
Epoch 11/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2509903616.0000 - mape: 19.5

In [57]:
print("Mean MAPE:",pred_df['MAPE'].mean())
import plotly.graph_objects as go
fig = go.Figure(data=[go.Histogram(x=pred_df['MAPE'], nbinsx=50)])
fig.update_layout(title='Histogram of Average MAPE', xaxis_title='Average MAPE', yaxis_title='Count')
fig.show()

Mean MAPE: 11.13422


In [58]:
fig = go.Figure()
fig.add_trace(go.Line(x = pred_df['Date'],
                    y = pred_df['MAPE'],
                    mode = 'lines+markers',
                    name = 'Average MAPE'))

fig.update_layout(xaxis_title = 'Date', yaxis_title = 'Average MAPE', title = 'Average MAPE vs Date')
fig.show()

c:\Users\milan\Desktop\CDIS Project\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


